In [1]:
import os
import re
import sys
import glob
import math
import logging
from pathlib import Path
from pprint import pprint

import numpy as np
import scipy as sp
import sklearn

import spacy
import tika
from tika import parser

%load_ext autoreload
%autoreload 2

import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
%config InlineBackend.figure_format = 'retina'

import seaborn as sns
sns.set_context("poster")
sns.set(rc={'figure.figsize': (16, 9.)})
sns.set_style("whitegrid")

import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

logging.basicConfig(level=logging.INFO, stream=sys.stdout)

In [2]:
from wb_nlp.processing import document

In [3]:
## Hints

# nlp = spacy.load('en_core_web_sm')

This notebook contains examples of how the `PDFDoc2Txt` class can be used to convert pdf documents into formatted text. Additional methods implemented in this class can also be applied to raw texts extracted from PDFs.

We start by creating an instance of the `PDFDoc2Txt`—`pdf_parser`.

In [4]:
pdf_parser = document.PDFDoc2Txt()

# Parsing a pdf file

Parsing a pdf file starts with the `parse` method. This method accepts a buffer of byte object or a string to a url or file path. The source type must be specified for the parser to correctly execute the processing.

Below is the implementation of the `parse` method. Tika is the main driver of the parser. We use the `xmlContent` flag to specify that we want to get an xml formatted output. The xml output contains relevant structure that we can leverage to generate an informed reconstruction of the document.

In [5]:
??pdf_parser.parse

Signature: pdf_parser.parse(source: Union[bytes, str], source_type: str = 'buffer') -> str
Source:   
    def parse(self, source: Union[bytes, str], source_type: str='buffer') -> str:
        """Parse a PDF document to text from different source types.

        Args:
            source:
                Source of the PDF that needs to be converted.
                The source could be a url, a path, or a buffer/file-like object
                to the PDF file.
            source_type:
                Specification of which source type is being provided in `source`.

        Returns:
            A string containing the parsed pdf file.

        """
        if source_type == 'url':
            buf = requests.get(source)
            pdf_text = parser.from_buffer(buf.content, xmlContent=True)

        elif source_type == 'file':
            pdf_text = parser.from_file(source, xmlContent=True)

        elif source_type == 'buffer':
            pdf_text = parser.from_buffer(source, xmlContent=

## Processing a single page

The xml returned by Tika contains page information captured by div tags. We used this to process documents by page.

The `process_page` method takes a tag element corresponding to the extracted page. Page level processing is then applied such as consolidation of paragraphs in the page and fixing footnote citations. We also perform concatenation of likely fragmented paragraphs.

In [6]:
??pdf_parser.process_page

Signature: pdf_parser.process_page(page: bs4.element.Tag) -> str
Docstring: <no docstring>
Source:   
    @staticmethod
    def process_page(page: bs4.element.Tag) -> str:
        paragraphs = []

        for p in page.find_all('p'):
            paragraph = PDFDoc2Txt.consolidate_paragraph(p.text)
            paragraph = PDFDoc2Txt.normalize_footnote_citations(paragraph)
            if not paragraph:
                continue

            prev_paragraph_end = paragraphs[-1][-1] if paragraphs and paragraphs[-1] else ''

            if prev_paragraph_end and (re.search(r'[a-zA-Z\-\,]', prev_paragraph_end) or (paragraph[0].islower())):  # .isalpha():
                paragraph = paragraphs[-1] + ' ' + paragraph
                paragraphs[-1] = paragraph
            else:
                paragraphs.append(paragraph)

        # for p in paragraphs:
        #     doc = self.nlp(p)
        #     self.sentences.extend(list(doc.sents))

        paragraphs = '\n\n'.join(paragraphs)
        return 

# Paragraph consolidation algorithm

The following method `consolidate_paragraph` contains the different heuristics for identifying fragmentation of paragraphs/sentences extracted from the pdf file.

This method is a static method allowing us to use this on arbitrary text document that may contain sentence level fragmentation due to OCR or other X-to-text conversion.

In [7]:
??pdf_parser.consolidate_paragraph

Signature:
pdf_parser.consolidate_paragraph(
    text_paragraph: str,
    min_fragment_len: int = 3,
) -> str
Source:   
    @staticmethod
    def consolidate_paragraph(text_paragraph: str, min_fragment_len: int=3) -> str:
        """Consolidate a `text_paragraph` with possible multiple newlines into one logical paragraph.

        Tika provides access to extracted text by paragraph. These paragraphs, however, may contain
        multiple newlines that break the paragraph arbitrarily. This function implements some heuristics
        to recover a logical representation of the paragraph.

        Args:
            text_paragraph:
                Text corresponding to an extracted paragraph from Tika.

        Returns:
            A string corresponding to a logical paragraph.

        """
        replace_chars = {'’': "'", '“': '"', '”': '"'}
        line_seps = set([' ', '-'])
        line_groups = [[]]

        text_paragraph = text_paragraph.replace('\r', '')

        lines = []
     

# Example

### Run tika docker image first.

https://hub.docker.com/r/apache/tika

```
sudo docker pull apache/tika
sudo docker run -d -p 9998:9998 apache/tika
```

The WB Docs repository contains pdf and txt versions of documents. However, some text versions are not formatted properly.

In [129]:
import requests

url = 'http://documents1.worldbank.org/curated/en/735931527600661308/text/126663-WP-PUBLIC-P164538-Malawi-Economic-Monitor-7-Realizing-Safety-Nets-Potential.txt'
txt_original = requests.get(url).content.decode('utf-8')

# pdf_url = 'http://documents1.worldbank.org/curated/en/735931527600661308/pdf/126663-WP-PUBLIC-P164538-Malawi-Economic-Monitor-7-Realizing-Safety-Nets-Potential.pdf'

pdf_url = 'https://openknowledge.worldbank.org/bitstream/handle/10986/34013/Designing-a-Credit-Facility-for-Women-Entrepreneurs-Lessons-from-the-Ethiopia-Women-Entrepreneurship-Development-Project.pdf?sequence=4&isAllowed=y'
txt_parsed = pdf_parser.parse(source=pdf_url, source_type='url')

In [130]:
buffer = requests.get(pdf_url).content
xml = document.parser.from_buffer(buffer, xmlContent=True)['content']

In [131]:
xmlB = document.BeautifulSoup(xml, features='html.parser')

In [286]:
nlp = spacy.load('en_core_web_sm')

Cleaning process:

- If coming from pdf, parse with tika to convert into xml using xmlContent=True.
- Process text per page. If page contains very few or no sentences, we could drop it since it may be a page full of tables or other details.
- Expand acronyms.
- Check length of text.
- Lemmatize.
- Remove noise.
- Detect language.
- Spell check.
- Respeller.
- Plural-singular map?
- Create recognizers of entities, e.g., countries, names, places, etc.





In [468]:
import importlib
import wb_nlp.extraction.extractor as extractor

importlib.reload(extractor)
ce = extractor.CountryExtractor(nlp, lower=True)

In [555]:
from wb_nlp.cleaning import cleaner
importlib.reload(cleaner)
lda_cleaner = cleaner.LDACleaner()

In [540]:
from enchant.checker import SpellChecker 

# https://spacy.io/api/annotation
POS_TAGS = ['POS', 'ADJ', 'ADP', 'ADV', 'AUX', 'CONJ', 'CCONJ', 'DET', 'INTJ', 'NOUN', 'NUM', 'PART', 'PRON', 'PROPN', 'PUNCT', 'SCONJ', 'SYM', 'VERB', 'X', 'SPACE']

LDA_POS_TAGS = [
    'ADJ', 'NOUN', 'PROPN', 'VERB'
]

LDA_INVALID_ENT_TYPE = [
    'GPE', 'COUNTRY', 'PERSON','ORG',
    'DATE', 'TIME', 'PERCENT', 'MONEY', 'QUANTITY',
    'ORDINAL',
    'CARDINAL',
]

EMBEDDING_POS_TAGS = [
    'ADJ',
    # 'ADP', 'ADV', 'AUX', 'CONJ', 'CCONJ', 'DET', 'INTJ',
    # 'NUM', 'PART',
    'NOUN', 'PROPN', 'VERB'
]

EMBEDDING_INVALID_ENT_TYPE = [
    # 'DATE',
    'TIME', 'PERCENT', 'MONEY',
    # 'QUANTITY',
    # 'ORDINAL',
    'CARDINAL',
]

en = SpellChecker("en_US")

def clean_text(text, pos_tag, exclude_ent_type, reverse=True):    
    # text = xmlB.find_all('div', attrs={'class': 'page'})[page_num].text
    text = (
        text
        .replace('\n', ' ')
        .replace('’', "'")
        .replace('“', '"')
        .replace('”', '"')
    )

    text = re.sub('\s+', ' ', text).strip().lower()

    doc = nlp(text)
    doc = ce(doc)

    tokens = []

    for t in doc:
        condition = (
            t.ent_type_ not in exclude_ent_type and
            t.pos_ in pos_tag and
            len(t) > 2 and
            t.is_alpha)

        if condition:

            # if not en.check(t.text):
            print((t.text, t.lemma_, t.pos_, t.ent_type_, t.ent_iob_, t._.normalized))

            # tokens.append(t.lemma_)

In [563]:
%%time

ct = lda_cleaner.clean_text(xmlB.text)

CPU times: user 7.15 s, sys: 1.16 s, total: 8.31 s
Wall time: 8.37 s


In [566]:
ct[:100]

['document',
 'credit',
 'facility',
 'woman',
 'entrepreneur',
 'ay',
 'lesson',
 'woman',
 'entrepreneurship',
 'development',
 'project',
 'lic',
 'th',
 'ri',
 'lic',
 'th',
 'ri',
 'lic',
 'th',
 'ri',
 'lic',
 'th',
 'ri',
 'ze',
 'report',
 'study',
 'collaborative',
 'effort',
 'current',
 'former',
 'team',
 'leader',
 'task',
 'team',
 'member',
 'report',
 'senior',
 'operation',
 'officer',
 'financial',
 'sector',
 'specialist',
 'senior',
 'financial',
 'sector',
 'economist',
 'analyst',
 'contribution',
 'background',
 'piece',
 'senior',
 'economist',
 'economist',
 'report',
 'guidance',
 'practice',
 'manager',
 'lead',
 'economist',
 'writing',
 'introductory',
 'section',
 'report',
 'overall',
 'editing',
 'consultant',
 'ay',
 'introduction',
 'government',
 'woman',
 'entrepreneurship',
 'development',
 'project',
 'aim',
 'earning',
 'employment',
 'growth',
 'micro',
 'small',
 'enterprise',
 'woman',
 'entrepreneur',
 'woman',
 'entrepreneur',
 'line',
 'cred

In [643]:
%%time
doc = nlp(text)
# doc = nlp(xmlB.text)

CPU times: user 88.5 ms, sys: 10.4 ms, total: 98.9 ms
Wall time: 98.7 ms


In [668]:
%%time

comp_nouns = []
curr_noun = []
for t in doc:
    print(t.text, t.lemma_, t.ent_type_, t.pos_)

    if not (t.is_alpha and len(t) >= 3):
        if t.text == '-':
            continue
        
        if len(curr_noun) > 1:
            comp_nouns.append('_'.join(curr_noun))

        curr_noun = []
        continue

    # if not curr_noun and t.pos_
    if t.pos_ in ['ADJ', 'NOUN']:
        curr_noun.append(t.lemma_ if t.lower_ != 'data' else t.lower_)
    else:
        if len(curr_noun) > 1:
            comp_nouns.append('_'.join(curr_noun))
        curr_noun = []

D D  PROPN
i i  NOUN
s s  VERB
r r  NOUN
u u  NOUN
p p  NOUN
t t  NOUN
i i  NOUN
ve ve  PROPN
F F  PROPN
i i  PROPN
n n  CCONJ
a a  DET
n n  ADJ
ce ce  NOUN
U u  NOUN
s s  NOUN
i i  NOUN
n n  CCONJ
g g  NOUN
P P  PROPN
syc syc  PROPN
h h  NOUN
o o  NOUN
m m  NOUN
et et  NOUN
r r  NOUN
i i  NOUN
c c  NOUN
s s  VERB
to to  ADP
O o  INTJ
ve ve  PROPN
rco rco  PROPN
m m  PROPN
e e  PROPN
Co Co  PROPN
l l  PROPN
l l  PROPN
ate ate  PROPN
ra ra  PROPN
l l  PROPN
Co Co  PROPN
n n  PROPN
st st  PROPN
ra ra  PROPN
i i  PRON
nt nt  PART
s s  VERB
i i  NOUN
n n  CCONJ
Et Et  PROPN
h h  NOUN
i i  NOUN
o o  NOUN
p p  NOUN
i i  NOUN
a50 a50  NOUN
5 5 CARDINAL NUM
Disruptive Disruptive  PROPN
Finance finance  NOUN
: :  PUNCT
Using use  VERB
Psychometrics Psychometrics  PROPN
to to  PART
Overcome overcome ORG VERB
Collateral Collateral ORG PROPN
Constraints Constraints ORG PROPN
in in  ADP
Ethiopia Ethiopia GPE PROPN
February February DATE PROPN
2019 2019 DATE NUM
The the  DET
Situation situation  NOU

In [670]:
comp_nouns[:1000]

['past_decade',
 'high_economic_growth',
 'private_sector',
 'survey_access',
 'main_business_environment_constraint',
 'woman_entrepreneur',
 'Most_growth',
 'woman_entrepreneur',
 'commercial_bank',
 'microfinance_situation',
 'high_minimum_loan_size',
 'excessive_collateral_constraint',
 'commercial_bank',
 'microfinance_institution',
 'asset_collateral',
 'micro_firm',
 'group_lending_scheme',
 'small_loan',
 'business_growth',
 'investment_lending_operation',
 'middle_challenge',
 'small_enterprise',
 'microfinance_institution',
 'female_entrepreneur',
 'technical_assistance',
 'provider_large',
 'individual_loan',
 'woman_entrepreneur',
 'good_practice',
 'individual_lending',
 'great_number',
 'woman_entrepreneur',
 'individual_loan',
 'psychometric_credit_assessment_tool',
 'next_step',
 'microfinance_institution',
 'good_partner',
 'careful_assessment',
 'active_borrower',
 'group_lending_market',
 'individual_lending',
 'new_product_line',
 'individual_loan',
 'lending_portfo

In [649]:
comp_nouns[:1000]

['area_provision_credit',
 'survey_access_finance',
 'business_environment_constraint',
 'medium_enterprise',
 'finance_challenge',
 'woman_entrepreneur',
 'woman_entrepreneur',
 'middle_trap',
 'microfinance_situation',
 'loan_size',
 'woman_access_loan',
 'bank_microfinance_institution',
 'asset_collateral',
 'micro_firm',
 'group_lending_scheme',
 'business_growth',
 'investment_lending_operation',
 'employment_female',
 'microfinance_institution',
 'loan_growth',
 'woman_entrepreneur',
 'number_woman_entrepreneur',
 'credit_assessment_tool',
 'microfinance_institution',
 'group_lending_market',
 'product_line',
 'lending_portfolio',
 'appraisal_mechanism',
 'asset_collateral',
 'house_building',
 'pilot_project_opportunity',
 'group_borrower',
 'credit_assessment',
 'group_borrower',
 'exhibit_name',
 'usd_tomorrow',
 'climate_change',
 'kilogram_rice',
 'purchasing_power_parity']

In [626]:
%%time

from spacy.matcher import Matcher
matcher = Matcher(nlp.vocab)

pattern = [{'POS': 'NOUN'}, {'POS': 'NOUN', 'OP': '+'}]
matcher.add('NOUN_SEQUENCE', None, pattern)


CPU times: user 69 µs, sys: 1e+03 ns, total: 70 µs
Wall time: 72 µs


In [627]:
%%time
m = matcher(doc)

d = pd.DataFrame(m, columns=['id', 'start', 'end'])
d = d.sort_values(['end', 'start'], ascending=[False, True]).drop_duplicates('end').drop_duplicates('start')
nn = []
for _, row in d.iterrows():
    nn.append(doc[row['start']: row['end']])
len(nn)

CPU times: user 273 ms, sys: 8.19 ms, total: 282 ms
Wall time: 283 ms


2316

In [597]:
doc[3:8]

r u p t i

CPU times: user 8.79 ms, sys: 273 µs, total: 9.07 ms
Wall time: 8.82 ms


33

33

In [583]:
%%time
page_num = 49
text = re.sub('\s+', ' ', txt_parsed[page_num].replace('\n', ' '))
text += ' tomorrow we see climate change in January 2019 of 2020 with 10 kilograms of rice. We also expect that the private-public partnership in this project will work. We think that the purchasing power parity of the country is low.'
# text = re.sub('\s+', ' ', xmlB.text.replace('\n', ' ').replace('’', "'").replace('“', '"').replace('”', '"')).strip()

clean_text(text, EMBEDDING_POS_TAGS, EMBEDDING_INVALID_ENT_TYPE, reverse=False)

('syc', 'syc', 'PROPN', '', 'O', '')
('rco', 'rco', 'PROPN', 'FAC', 'B', '')
('ate', 'ate', 'PROPN', '', 'O', '')
('disruptive', 'disruptive', 'ADJ', '', 'O', '')
('finance', 'finance', 'NOUN', '', 'O', '')
('using', 'use', 'VERB', '', 'O', '')
('psychometrics', 'psychometric', 'NOUN', '', 'O', '')
('overcome', 'overcome', 'VERB', '', 'O', '')
('collateral', 'collateral', 'ADJ', '', 'O', '')
('constraints', 'constraint', 'NOUN', '', 'O', '')
('ethiopia', 'ethiopia', 'PROPN', 'COUNTRY', 'B', 'Ethiopia')
('february', 'february', 'PROPN', 'DATE', 'B', '')
('situation', 'situation', 'NOUN', '', 'O', '')
('ethiopia', 'ethiopia', 'PROPN', 'COUNTRY', 'B', 'Ethiopia')
('past', 'past', 'ADJ', 'DATE', 'I', '')
('decade', 'decade', 'NOUN', 'DATE', 'I', '')
('ethiopia', 'ethiopia', 'PROPN', 'COUNTRY', 'B', 'Ethiopia')
('achieved', 'achieve', 'VERB', '', 'O', '')
('high', 'high', 'ADJ', '', 'O', '')
('economic', 'economic', 'ADJ', '', 'O', '')
('growth', 'growth', 'NOUN', '', 'O', '')
('establishin

In [480]:
text = re.sub('\s+', ' ', xmlB.find_all('div', attrs={'class': 'page'})[page_num].text.replace('\n', ' ').replace('’', "'").replace('“', '"').replace('”', '"')).strip()
text

'B ette r Lo a n s o r B ette r B o r rowe r s? I m p a ct o f M e s o - C re d i t o n Fe m a l e - O w n e d E nte r p r i s e s i n Et h i o p i a44 IMPACT OF WEDP LOANS 25 Figure 3. Probability of receiving a loan based on observable characteristics Figure 4. Probit regression of propensity score and test of balancing property Probit Regression of Propensity Score Figure 4. Probit Regression of Propensity Score and Test of Balancing Property'

addition, 
experience suggested that pull‐down menus with fixed choices simplified data entry and minimized 
likelihood of errors. 

In the initial phase, tests were run on the database to ensure data quality. The database could be 
exported to Excel and then used with any statistical software. This was helpful for detecting errors in 
the  system  and  refining  data  entry  procedures.  Moreover,  WEDP  M&E  staff  used  exported  data 
regularly to conduct advanced analysis. Although the built‐in reporting functions shown in the previous 
section worked well for basic monitoring and provided some standard evaluation tools, the richness 
of the database could be used in a range of ways when not restricted to prespecified parameters.  

The WEDP also had a database expert conduct in‐depth advanced data quality assessments and check 
for errors,8 which provided useful technical inputs for refining the system.  

                                                            
8 It was nece

In [483]:
xmlB.find_all('div', attrs={'class': 'page'})[page_num]

<div class="page"><p></p>
<p>B ette r  Lo a n s  o r  B ette r  B o r rowe r s?  I m p a ct  o f  M e s o - C re d i t  o n  Fe m a l e - O w n e d  E nte r p r i s e s  i n  Et h i o p i a44
</p>
<p>IMPACT OF WEDP LOANS 25
</p>
<p>Figure 3. Probability of receiving a loan based on observable characteristics
</p>
<p>Figure 4. Probit regression of propensity score and test of balancing property
</p>
<p>Probit Regression of Propensity Score
Figure 4. Probit Regression of Propensity Score and  
</p>
<p>Test of Balancing Property</p>
<p></p>
</div>

In [456]:
page_num = 49
text = re.sub('\s+', ' ', xmlB.find_all('div', attrs={'class': 'page'})[page_num].text.replace('\n', ' ').replace('’', "'").replace('“', '"').replace('”', '"')).strip()
doc1 = nlp(text)

docs1 = [s for s in doc1.sents]

# for s in docs1:
#     print(s)
#     print('------')

text = re.sub('\s+', ' ', txt_parsed[page_num].replace('\n', ' '))
# doc2 = nlp(re.sub('\s+', ' ', txt_parsed[page_num].replace('\n', ' ')))
doc2 = nlp(text)

docs2 = [s for s in doc2.sents]

# for s in docs2:
#     print(s)
#     print('------')

# len(docs1) == len(docs2)

# np.where(np.array([d1.text.strip() == d2.text.strip() for d1, d2 in zip(docs1, docs2)]) != True)

%time countries = [(t._.normalized, t._.is_country, t.lemma_, t.ent_type_, t.pos_, t._.code) for t in ce(doc2) if t._.is_country]
countries

CPU times: user 2.98 ms, sys: 2 µs, total: 2.98 ms
Wall time: 2.99 ms


[('Ethiopia', True, 'Ethiopia', 'COUNTRY', 'PROPN', 'ETH'),
 ('Ethiopia', True, 'Ethiopia', 'COUNTRY', 'PROPN', 'ETH'),
 ('Ethiopia', True, 'Ethiopia', 'COUNTRY', 'PROPN', 'ETH'),
 ('Ethiopia', True, 'Ethiopia', 'COUNTRY', 'PROPN', 'ETH'),
 ('Ethiopia', True, 'Ethiopia', 'COUNTRY', 'PROPN', 'ETH'),
 ('Ethiopia', True, 'Ethiopia', 'COUNTRY', 'PROPN', 'ETH'),
 ('Ethiopia', True, 'Ethiopia', 'COUNTRY', 'PROPN', 'ETH'),
 ('Ethiopia', True, 'Ethiopia', 'COUNTRY', 'PROPN', 'ETH'),
 ('Ethiopia', True, 'Ethiopia', 'COUNTRY', 'PROPN', 'ETH'),
 ('Ethiopia', True, 'Ethiopia', 'COUNTRY', 'PROPN', 'ETH'),
 ('Ethiopia', True, 'Ethiopia', 'COUNTRY', 'PROPN', 'ETH')]

In [457]:
doc2

D i s r u p t i ve F i n a n ce U s i n g P syc h o m et r i c s to O ve rco m e Co l l ate ra l Co n st ra i nt s i n Et h i o p i a50 5 Disruptive Finance: Using Psychometrics to Overcome Collateral Constraints in Ethiopia February 2019 The Situation in Ethiopia Over the past decade, Ethiopia has achieved high economic growth, establishing the country among the fastest growing economies both in Africa and the developing world. However, Ethiopia is falling behind its peers in the area of provision of credit to the private sector. According to the World Bank's Enterprise Surveys access to finance is perceived as the main business environment constraint by micro (41%), small (36%) and medium (29%) enterprises in Ethiopia, compared to a Sub-Saharan average of 24%, 20% and 16% respectively.13 Access to finance is a challenge for men and women alike but difficulties are amplified for women, who are less likely to own assets, and opportunities for women entrepreneurs in Ethiopia lag far beh

In [288]:
ix = 29
docs1[ix - 1], docs2[ix]

(At the end of 2017, after securing external financing from Italy (USD 15.8 million) and Japan (USD 50 million), WEDP entered its second phase, expanding into 4 additional cities,4 adopting new technologies and innovations, and consolidating its interventions.,
 _3 Shortly thereafter, a WEDP revolving fund was approved to replenish the line of credit from repaid principals.)

In [289]:
for e, s in enumerate(doc2.sents):
    # print(s)
    # print('------')
    if e == ix:
        break

In [290]:
t = s[0]
s

_3 Shortly thereafter, a WEDP revolving fund was approved to replenish the line of credit from repaid principals.

CPU times: user 5.06 ms, sys: 48 µs, total: 5.11 ms
Wall time: 5.14 ms


[('Ethiopia', True, 'Ethiopia', 'COUNTRY', 'PROPN', 'ETH'),
 ('Ethiopia', True, 'Ethiopia', 'COUNTRY', 'PROPN', 'ETH'),
 ('Peru', True, 'Peru', 'COUNTRY', 'PROPN', 'PER'),
 ('Mexico', True, 'Mexico', 'COUNTRY', 'PROPN', 'MEX'),
 ('Guatemala', True, 'Guatemala', 'COUNTRY', 'PROPN', 'GTM'),
 ('Ecuador', True, 'Ecuador', 'COUNTRY', 'PROPN', 'ECU'),
 ('India', True, 'India', 'COUNTRY', 'PROPN', 'IND'),
 ('Indonesia', True, 'Indonesia', 'COUNTRY', 'PROPN', 'IDN'),
 ('Kenya', True, 'Kenya', 'COUNTRY', 'PROPN', 'KEN'),
 ('Ghana', True, 'Ghana', 'COUNTRY', 'PROPN', 'GHA'),
 ('Ethiopia', True, 'Ethiopia', 'COUNTRY', 'PROPN', 'ETH'),
 ('Ethiopia', True, 'Ethiopia', 'COUNTRY', 'PROPN', 'ETH'),
 ('Ethiopia', True, 'Ethiopia', 'COUNTRY', 'PROPN', 'ETH'),
 ('Japan', True, 'Japan', 'COUNTRY', 'PROPN', 'JPN'),
 ('Italy', True, 'Italy', 'COUNTRY', 'PROPN', 'ITA'),
 ('Canada', True, 'Canada', 'COUNTRY', 'PROPN', 'CAN')]

[('Ethiopia', True, 'Ethiopia', 'COUNTRY', 'PROPN', 'ETH'),
 ('Ethiopia', True, 'Ethiopia', 'COUNTRY', 'PROPN', 'ETH'),
 ('Peru', True, 'Peru', 'COUNTRY', 'PROPN', 'PER'),
 ('Mexico', True, 'Mexico', 'COUNTRY', 'PROPN', 'MEX'),
 ('Guatemala', True, 'Guatemala', 'COUNTRY', 'PROPN', 'GTM'),
 ('Ecuador', True, 'Ecuador', 'COUNTRY', 'PROPN', 'ECU'),
 ('India', True, 'India', 'COUNTRY', 'PROPN', 'IND'),
 ('Indonesia', True, 'Indonesia', 'COUNTRY', 'PROPN', 'IDN'),
 ('Kenya', True, 'Kenya', 'COUNTRY', 'PROPN', 'KEN'),
 ('Ghana', True, 'Ghana', 'COUNTRY', 'PROPN', 'GHA'),
 ('Ethiopia', True, 'Ethiopia', 'COUNTRY', 'PROPN', 'ETH'),
 ('Ethiopia', True, 'Ethiopia', 'COUNTRY', 'PROPN', 'ETH'),
 ('Ethiopia', True, 'Ethiopia', 'COUNTRY', 'PROPN', 'ETH'),
 ('Japan', True, 'Japan', 'COUNTRY', 'PROPN', 'JPN'),
 ('Italy', True, 'Italy', 'COUNTRY', 'PROPN', 'ITA'),
 ('Canada', True, 'Canada', 'COUNTRY', 'PROPN', 'CAN')]

In [448]:
doc2

5M ay 2 02 0 As of December 2019, approximately 36,000 women entrepreneurs registered for WEDP. Of these, ETB 3.96 billion (USD 158.1 million) _5 has been disbursed to 13,870 clients, of which 59.8 percent (ETB 2.37 billion) were from donor funds, while the MFI internal funds (ETB 1.05 billion) and revolving fund (ETB 540 million) comprised the remaining 26.6 percent and 13.6 percent, respectively. Moreover, a total of 20,744 women entrepreneurs have participated in entrepreneurial trainings across 10 cities, with a 98.6 percent completion rate. Demand for loans and training shows little sign of abating, as women entrepreneurs from across various sectors, including trade, services, manufacturing, construction and agriculture, continue to register for the project. WEDP has been widely recognized for its achievements in empowering women entrepreneurs and for raising their profile throughout Ethiopia's financial system. The project model has demonstrated how an incentives-based strategy, 

In [292]:
%%time

import pkg_resources
from symspellpy.symspellpy import SymSpell

# Set max_dictionary_edit_distance to avoid spelling correction
sym_spell = SymSpell(max_dictionary_edit_distance=0, prefix_length=7)
dictionary_path = pkg_resources.resource_filename(
    "symspellpy", "frequency_dictionary_en_82_765.txt")
# term_index is the column of the term and count_index is the
# column of the term frequency
sym_spell.load_dictionary(dictionary_path, term_index=0, count_index=1)

# a sentence without any spaces
# input_term = "thequickbrownfoxjumpsoverthelazydog"
input_term = re.sub('\s+', '', text)
result = sym_spell.word_segmentation(input_term)
# print("{}, {}, {}".format(result.corrected_string, result.distance_sum,
#                           result.log_prob_sum))

CPU times: user 1.37 s, sys: 53.8 ms, total: 1.43 s
Wall time: 1.57 s


In [296]:
print(result.corrected_string)

De signing aCr edit Facility for Wo men Ent rep rene ursLes sons from the EthiopiaWo men Ent rep rene ur ship D eve lop men tProject(WEDP)4WEDP loans began disbursing in J anu ary2014 through 12 partner micro finance institutions (MFIs) across 6 targeted cities in Ethi-opia.1By the end of the calendar year , the project had issued ETB456.6 million (USD23.3 million )to1,863 women entrepreneurs – over half the dedicated line of credit .2WEDP clients represented abroad spectrum of Ethiopia's business community , ranging from retail stores to restaurants to beauty salons .A mong those who borrowed ,66 percent were first - time borrowers , and yet repayment rates were healthy , standing at 99.1 percent .In tandem ,3,083 women had participated in entrepreneurship trainings .By mid -2015, the high demand for credit had led to a rapid depletion of WEDP funds , prompting the project leadership to explore additional sources of financing .In light of the declining balance , partner MFIs began dis

In [280]:
s = nlp('repaid principal is beng given to the authority')
print(s._.performed_spellCheck) 
[(t.text, t.lemma_, t.pos_, t.ent_type_, t.ent_iob_) for t in s]

True


[('repaid', 'repay', 'VERB', '', 'O'),
 ('principal', 'principal', 'NOUN', '', 'O'),
 ('is', 'be', 'AUX', '', 'O'),
 ('beng', 'beng', 'NOUN', 'PERSON', 'B'),
 ('given', 'give', 'VERB', '', 'O'),
 ('to', 'to', 'ADP', '', 'O'),
 ('the', 'the', 'DET', '', 'O'),
 ('authority', 'authority', 'NOUN', '', 'O')]

# Result from direct txt version

In [36]:
print(txt_original[12850:20000])

MALAWI ECONOMIC MONITOR MAY 2018
OVERVIEW
                                                          challenges related to erratic energy and water
The Malawi Economic Monitor (MEM) provides an
                                                          supply, which had a particularly negative impact
analysis of economic and structural development
                                                          on      manufacturing.    Within  services,    the
issues in Malawi. This edition was published in May
                                                          performance of the wholesale and retail trade and
2018. It follows on from the six previous editions of
                                                          distribution sub-sectors declined as a result of
the MEM and is part of an ongoing series, with
                                                          subdued domestic demand.
future editions to follow twice per year.
                                                 

# Result from PDFDoc2Txt

In [39]:
print(txt[4])

1 « MALAWI ECONOMIC MONITOR MAY 2018

OVERVIEW The Malawi Economic Monitor (MEM) provides an analysis of economic and structural development issues in Malawi. This edition was published in May 2018. It follows on from the six previous editions of the MEM and is part of an ongoing series, with future editions to follow twice per year.

The aim of the publication is to foster better- informed policy analysis and debate regarding the key challenges that Malawi faces in its endeavor to achieve high rates of stable, inclusive and sustainable economic growth.

The MEM consists of two parts: Part 1 presents a review of recent economic developments and a macroeconomic outlook. Part 2 focuses on a special selected topic relevant to Malawi's development prospects.

In this edition, the special topic focuses on Social Safety Nets. This is a defining moment for Malawi to transform its safety net. The recently approved second Malawi National Social Support Program (MNSSP II) works towards the creat

## Observations

Using the text version from the WB Docs repository shows a fragmented structure. Columnar flows in a pdf page are literally placed side-by-side. This proves to be a challenge since inferring of sentences is not straighforward as simply replacing line breaks with spaces.

On the other hand, we can see that the `PDFDoc2Txt` has managed to recover the logical sentences in the text. It was able to properly concatenate fragmented sentences and also identified which correspond to a single column flow.